# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [ ]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

## Consumer Price Index

### OECD Consumer Price Index

In [ ]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


### FRED Consumer Price Index

In [ ]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

# humblCOMPASS 

In [2]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2022-03-01",
    end_date="2023-02-16",
    membership="humblPEON"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
compass = await toolbox.fundamental.humbl_compass(
    country="united_states",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (async)


DEBUG: HumblCompassFetcher || Running .transform_query()
INFO: HumblCompassFetcher || CLI start date: 2021-11-01 and CPI start date: 2021-11-01. Dates are adjusted to account for CLI data release lag and z-score calculation window.
DEBUG: HumblCompassFetcher || Running .extract_data()
INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: https://data.humblfinance.io/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2021-11-01&end_date=2023-02-16&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || Fetching data from: https://data.humblfinance.io/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2021-11-01&end_date=2023-02-16&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 3.6494s
INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: https://data.humblfinanc

In [4]:
compass.to_polars()

date_month_start,country,cpi,cpi_3m_delta,cli,cli_3m_delta,humbl_regime
date,str,f64,f64,f64,f64,str
2022-02-01,"""United States""",7.87,1.06,100.83,-0.29,"""humblBLOAT"""
2022-03-01,"""United States""",8.54,1.51,100.61,-0.45,"""humblBLOAT"""
2022-04-01,"""United States""",8.26,0.78,100.34,-0.63,"""humblBLOAT"""
2022-05-01,"""United States""",8.58,0.71,100.01,-0.81,"""humblBLOAT"""
2022-06-01,"""United States""",9.06,0.52,99.69,-0.93,"""humblBLOAT"""
…,…,…,…,…,…,…
2022-09-01,"""United States""",8.2,-0.86,99.03,-0.65,"""humblBUST"""
2022-10-01,"""United States""",7.75,-0.78,98.9,-0.51,"""humblBUST"""
2022-11-01,"""United States""",7.11,-1.15,98.8,-0.4,"""humblBUST"""


In [7]:
compass

HumblObject

id: 0682e47c-244c-7910-8000-39c427bc031a
results: b"ARROW1\x00\x00\xff\xff\xff\xff \x02\x00\x00\x04\x00\x00\x00\xf2\xff\xff\...
equity_data: None
provider: yfinance
warnings: [{'category': 'HumblDataWarning', 'message': 'Start date adjusted to 2023...
chart: [{'content': '{"data":[{"hovertemplate":"\\u003cb\\u003e%{text}\\u003c\\u002...
extra: {'humbl_regime_recommendations': {'asset_classes': {'best': [<AssetRecommend...
context_params: {'symbols': 'AAPL', 'interval': '1d', 'start_date': datetime.date(2...
command_params: {'country': 'united_states', 'cli_start_date': '2023-01-17', 'cpi_s...
backtest: <function Fundamental.humbl_compass.<locals>.backtest at 0x137929a80>

In [34]:
compass.show()

In [19]:
compass.warnings

[Warning_(category='HumblDataWarning', message='Start date adjusted to 2022-12-02 based on humblPEON membership (2Y of data).'),
 Warning_(category='HumblDataWarning', message='Z-score defaulted to None. No z-score data will be calculated.')]

In [2]:
from aiocache import RedisCache


await RedisCache(namespace="humbl_compass").clear()

True

In [14]:
%load_ext autoreload
%autoreload 2

import polars as pl
from humbldata.core.standard_models.openbbapi.EconomyCompositeLeadingIndicatorQueryParams import EconomyCompositeLeadingIndicatorQueryParams
from humbldata.core.utils.openbb_api_client import OpenBBAPIClient


cli_query_params = EconomyCompositeLeadingIndicatorQueryParams(
    start_date="2022-08-02",
    end_date="2024-11-01",
    provider="oecd",
    country="united_states",
)
api_client = OpenBBAPIClient()
cli_response = await api_client.fetch_data(
    obb_path="economy.composite_leading_indicator",
    api_query_params=cli_query_params,
)
oecd_cli_data = cli_response.to_polars(collect=False)
oecd_cli_data = oecd_cli_data.rename({"value": "cli"}).with_columns(
    [pl.col("date").dt.month_start().alias("date_month_start")]
)

INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 0.1986s


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from humbldata.core.standard_models.openbbapi.EquityPriceHistoricalQueryParams import (
    EquityPriceHistoricalQueryParams,
)


equity_query_params = EquityPriceHistoricalQueryParams(
    symbol=["AAPL", "AMD"],
    start_date="2022-12-02",
    end_date="2024-11-01",
    provider="yfinance"
)

api_client = OpenBBAPIClient()
equity_response = await api_client.fetch_data(
    obb_path="equity.price.historical",
    api_query_params=equity_query_params,
)
equity_data = equity_response.to_polars(collect=False)


INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/equity/price/historical?provider=yfinance&symbol=AAPL&symbol=AMD&start_date=2022-12-02&end_date=2024-11-01
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/equity/price/historical?provider=yfinance&symbol=AAPL&symbol=AMD&start_date=2022-12-02&end_date=2024-11-01


INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 0.2545s


In [27]:
equity_data.collect()

date,open,high,low,close,volume
date,f64,f64,f64,f64,i64
2022-12-02,75.050003,76.279999,73.650002,74.980003,64377200
2022-12-05,74.540001,75.169998,72.82,73.620003,47022200
2022-12-06,73.080002,73.389999,69.400002,70.269997,64965200
2022-12-07,69.529999,71.370003,69.279999,70.139999,59180100
2022-12-08,70.139999,71.220001,69.0,70.470001,56734100
…,…,…,…,…,…
2024-10-28,158.5,160.279999,157.039993,159.919998,36010100
2024-10-29,161.100006,167.509995,158.940002,166.25,71896000
2024-10-30,153.009995,153.119995,148.100006,148.600006,87701700
